In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 70.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import json
import os
import tensorflow as tf
import onnx

In [ ]:
# Save tensor as binary file
def save_tensor_as_bin(tensor, filepath):
    np_array = tensor.numpy() if isinstance(tensor, tf.Tensor) else tensor
    np_array.tofile(filepath)

# Save JSON file
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

# Create directories
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [ ]:
import numpy as np
import json
import os

# Helper function to create directories if they do not exist
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Helper function to save tensor to a .txt file (for weights)
def save_tensor_as_txt(tensor, file_path):
    with open(file_path, "w") as f:
        # Reshape the tensor to 2D before saving
        reshaped_tensor = tensor.reshape(-1, tensor.shape[-1])  # Flatten to 2D preserving last dimension
        # Convert the tensor to a NumPy array and write each element to the text file
        np.savetxt(f, reshaped_tensor, fmt='%0.8f')  # You can adjust the format if needed

# Helper function to save the attributes as a JSON file
def save_json(data, file_path):
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)

def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/input")
    create_dir(f"{save_dir}/output")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/reference")

    intermediate_outputs = {}  # To store outputs for sequential execution

    # Run input through the model layer by layer
    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor to file
        save_tensor_as_txt(x.numpy(), f"{save_dir}/input/{layer_name}_input.txt")  # Save as .txt

        # Save weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                save_tensor_as_txt(weight.numpy(), f"{save_dir}/weights/{layer_name}_{weight_name}.txt")  # Save weights as .txt

        # Compute layer output
        x = layer(x)

        # Save output tensor to file
        save_tensor_as_txt(x.numpy(), f"{save_dir}/output/{layer_name}_output.txt")  # Save as .txt

        # Save layer attributes
        attributes = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
        }
        if hasattr(layer, 'kernel_size'):
            attributes["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            attributes["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            attributes["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            attributes["activation"] = layer.activation.__name__

        save_json(attributes, f"{save_dir}/reference/{layer_name}_attributes.json")

        # Store intermediate outputs for sequential execution
        intermediate_outputs[layer_name] = x.numpy()

    return intermediate_outputs

In [ ]:
model = tf.keras.models.load_model("/content/cifar10_model1.h5")

In [ ]:
def read_input_matrix(file_path):
    # Read the file and convert to a NumPy array
    input_matrix = np.loadtxt(file_path).reshape((32, 32, 3))
    return input_matrix

# Example: Read the input from your file
input_matrix = read_input_matrix('/content/drive/MyDrive/image_matrix_4.txt')

# Convert the input to a TensorFlow tensor
input_tensor = tf.convert_to_tensor(input_matrix, dtype=tf.float32)
input_tensor = tf.expand_dims(input_tensor, axis=0)
layer_wise_dump_tf(model, input_tensor, save_dir="/content/drive/MyDrive/Data_new")

{'conv2d': array([[[[0.        , 0.        , 0.        , ..., 0.01312021,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.00040885, 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.00550007, 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.01809026, 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]],
 
         [[0.        , 0.        , 0.01757238, ..., 0.0163503 ,
           0.        , 0.        ],
          [0.        , 0.        , 0.00599828, ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.   

In [ ]:
import json
import tensorflow as tf

# Load your trained model
saved_model_path = '/content/cifar10_model1.h5'
model = tf.keras.models.load_model(saved_model_path)

def extract_model_config(model):
    model_config = {"layers": []}

    # Iterate through the layers and extract the necessary information
    for layer in model.layers:
        layer_info = {}
        layer_info["type"] = layer.__class__.__name__
        layer_info["name"] = layer.name  # Add layer name

        # Extract input and output shapes if they exist, set to None if not
        input_shape = list(layer.input_shape) if hasattr(layer, 'input_shape') else None
        output_shape = list(layer.output_shape) if hasattr(layer, 'output_shape') else None

        layer_info["input_shape"] = input_shape
        layer_info["output_shape"] = output_shape

        # Extract attributes for specific layer types
        if isinstance(layer, tf.keras.layers.Conv2D):
            layer_info["attributes"] = {
                "kernel_size": layer.kernel_size,
                "strides": layer.strides,
                "activation": layer.activation.__name__,
                "padding": layer.padding.upper(),
                "input_shape": input_shape,
                "output_shape": output_shape
            }
            # Extract weights (kernel and bias)
            layer_info["weights"] = []
            layer_info["weights"].append(layer.get_weights()[0].tolist())  # Kernel weights
            if len(layer.get_weights()) > 1:
                layer_info["weights"].append(layer.get_weights()[1].tolist())  # Bias weights

        elif isinstance(layer, tf.keras.layers.Dense):
            layer_info["attributes"] = {
                "units": layer.units,
                "activation": layer.activation.__name__,
                "input_shape": input_shape,
                "output_shape": output_shape
            }
            # Extract weights (kernel and bias)
            layer_info["weights"] = []
            layer_info["weights"].append(layer.get_weights()[0].tolist())  # Kernel weights
            if len(layer.get_weights()) > 1:
                layer_info["weights"].append(layer.get_weights()[1].tolist())  # Bias weights

        elif isinstance(layer, tf.keras.layers.MaxPooling2D):
            layer_info["attributes"] = {
                "pool_size": layer.pool_size,
                "strides": layer.strides,
                "input_shape": input_shape,
                "output_shape": output_shape
            }

        elif isinstance(layer, tf.keras.layers.BatchNormalization):
            layer_info["attributes"] = {
                "input_shape": input_shape,
                "output_shape": output_shape
            }

        elif isinstance(layer, tf.keras.layers.Dropout):
            layer_info["attributes"] = {
                "rate": layer.rate,
                "input_shape": input_shape,
                "output_shape": output_shape
            }

        # Add layer information to model config
        model_config["layers"].append(layer_info)

    return model_config


# Generate the model config as a JSON-like dictionary
model_config = extract_model_config(model)

# Convert the model config to JSON and save it to a file
json_file_path = '/content/drive/MyDrive/cifar_model_2/model_config_2.json'
with open(json_file_path, 'w') as json_file:
    json.dump(model_config, json_file, indent=4)

print(f"Model configuration saved to: {json_file_path}")

Model configuration saved to: /content/drive/MyDrive/cifar_model_2/model_config_2.json


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

# Load the .h5 model
 # Replace with your .h5 file path
model = tf.keras.models.load_model("/content/cifar10_model1.h5")

# Print a summary to find the batch normalization layers
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 128)         │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 562,316 (2.15 MB)

 Trainable params: 562,314 (2.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)